Name: Main_multithreader

Version: 1.0

Author: Anish Vasan

Organization: Chen/Eyckmans Lab, Boston University

Date: 20240815

## IMPORTS ##

In [1]:
import threading
import subprocess
from humanfriendly import format_timespan
from pathlib import Path
from woundcompute import image_analysis as ia
from wc_function_2_1 import *
import time

C:\Users\anish\.conda\envs\wound-compute-env\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## DEFINITIONS ##
## These require user input/changes based on experiments ##

In [2]:
image_type, is_pillars, parallels = yaml_input_gui()
is_pillars = bool(is_pillars)
is_fl = False #Options: True, False

## RUN CODE ##

**Section 1: Navigate to data folder containing .nd file**

In [9]:
print("Current config:")
print("Image Type:", image_type)
print("Fluorescent Images:", is_fl)
print("Track Pillars:", is_pillars)

path_input, path_output = io_function()

Current config:
Image Type: Ph1
Fluorescent Images: False
Track Pillars: True
Please open the directory that contains your .nd file
Inputed path: 


PermissionError: [WinError 5] Access is denied: '/'

**Section 2: Execute functions to prepare folder structure for woundcompute**

In [4]:
print(image_type)
# Create yaml file for image type
create_yaml(path_output, image_type, is_fl, is_pillars)
print("Created .yaml file")
# Create list of .nd files/experiments in the input folder
basename_list = define_basename_list(path_input)
print("Created basename list:", basename_list)

print("Sorting images into corresponding basename folders...")
# Sort images in input folder into Sorted/basename folder
sort_basename_folders(basename_list, path_input, path_output)
print("Completed!")

# Extract stage position information from .nd file
stage_pos_nd = extract_nd_info(basename_list,path_output)
print("Extracted stage position information from .nd files")

print("Sorting images in each basename folder into their corresponding stage position folders...")
#  Sort images in each basename folder into their corresponding stage position folders
sort_stage_pos_folders(basename_list,path_output, stage_pos_nd, image_type)

print("Dividing tissue folders into processing subfolders to maximize multicore performance...")
#Divide tissue folders into processing groups to maximize processing power
sort_parallel_processes(basename_list, path_output, stage_pos_nd, image_type, parallels)

print("Completed")

ph1
Created .yaml file
Created basename list: ['tissue_ai', 'tissue_ai1', 'tissue_bi']
Sorting images into corresponding basename folders...
Completed!
Extracted stage position information from .nd files
Sorting images in each basename folder into their corresponding stage position folders...
Dividing tissue folders into processing subfolders to maximize multicore performance...
Error: C:/Working Folder/20240713_R5BP14_FUNCAT/S2/tissue_ai/s06 - No such file or directory.
Error: C:/Working Folder/20240713_R5BP14_FUNCAT/S2/tissue_ai/s07 - No such file or directory.
Error: C:/Working Folder/20240713_R5BP14_FUNCAT/S2/tissue_ai/s08 - No such file or directory.
Error: C:/Working Folder/20240713_R5BP14_FUNCAT/S2/tissue_ai/s09 - No such file or directory.
Error: C:/Working Folder/20240713_R5BP14_FUNCAT/S2/tissue_ai/s10 - No such file or directory.
Error: C:/Working Folder/20240713_R5BP14_FUNCAT/S2/tissue_ai/s11 - No such file or directory.
Error: C:/Working Folder/20240713_R5BP14_FUNCAT/S2/tis

**Section 3: Execute woundcompute for all basename folders in the Sorted folder**

In [ ]:
print("Starting wound compute for each stage position...")
print("Start time:",time.ctime())
start = time.time()
for index, basename in enumerate(basename_list):
    time_all=[]
    for stage_pos in range(1, stage_pos_nd[index]+1):
        current = time.time()
        input_location = Path(path_output+'/'+basename+'/'+'s' + f"{stage_pos:02}")
        try:
            time_all, action_all = ia.run_all(input_location)
            secondsPassed = time.time()-current
            print("tissue number:", stage_pos, "time:", format_timespan(secondsPassed))
        except Exception as ex:
            time_all.append(time.time())
            print("tissue number:", stage_pos, "time:", time.ctime())
            print("---------ERROR OF SOME DESCRIPTION HAS HAPPENED-------")
            print(ex)
            print("------------------------------------------------------")

print("Completed! Hurray!")
print("End time:",time.ctime())
print("Total time taken:", format_timespan(time.time()-start))

print("Please run extract_dataset to consolidate outputs into an excel file!")